In [ ]:
import pandas as pd
import numpy as np
# from google.colab import files

In [ ]:
#defining bias and input values
n_bits = int(8)
n_bits_input = n_bits
num_inputs = 4           
is_signed = 1           #1= signed || 0= unsigned
is_sigmoid_signed = 1   #1= signed || 0= unsigned
weights_on_memory = 1   #1: on memory || 0: weights = inputs

m_bin = 0 # 'm' binary digits are the integer part
n_bin = n_bits - m_bin    # 'n' binary digits are the fractional part of `x`;
n_bin_sigmoid = n_bin #+ 3 #número de casas decimais sigmoide memória

download_files = 0      #1= para baixar || 0= não baixar

In [ ]:
# atenuacao_rescale = 0.95
atenuacao_rescale = 1

In [ ]:
range_tb_df = (2**m_bin)              #tamanho do dataframe do tb
#DX SIGMOID
is_signed = True           #1= signed || 0= unsigned
if is_signed:
  unsigned_offset = 0
  offset_df_memory = 0
  # norm_y = (2**(m_bin-1))                   # normalização em y: 256(8bits)
  norm_y = 1
  dx_sigmoid_final = 1/(2**(n_bin_sigmoid-1))     #dx da sigmoid na memória -> depende das casas .fixo com adicional para resolução de dx_input
  
else:
  unsigned_offset = int((2**m_bin)/2)  # 128
  offset_df_memory = unsigned_offset - ((sigmoid_range/2)*mult_factor)
  norm_y = (2**(m_bin))                   # normalização em y: 256(8bits)
  dx_sigmoid_final = 1/(2**(n_bin_sigmoid))     #dx da sigmoid na memória -> depende das casas .fixo com adicional para resolução de dx_input


dx_sigmoid = dx_sigmoid_final/mult_factor   #dx necessário antes da expansão

if (dx_sigmoid == 1):
  total_dx = range_tb_df+1
else:
  
  total_dx = (1/dx_sigmoid)*sigmoid_range +1

In [ ]:
range_tb_df

In [ ]:
sigmoid_range

In [ ]:
# range_tb_df
sigmoid_dx_list = np.linspace(-(sigmoid_range/2),(sigmoid_range/2),int(total_dx)) #vai de -128 a 127, dividido em 256 pontos (8 bits)
# sigmoid_dx_list

In [ ]:
# df_sigmoid = pd.DataFrame((range(-int(range_tb_df/2),int(range_tb_df/2)+1,1))) #start, end, step
df_sigmoid = pd.DataFrame(np.linspace(-(sigmoid_range/2),(sigmoid_range/2),int(total_dx))) #start, end, step
df_sigmoid.columns = ['n'] # INPUTS_df
# df_sigmoid['dx'] = sigmoid_dx_list
df_sigmoid = df_sigmoid*atenuacao_rescale
df_sigmoid.head()

In [ ]:
# ((1/(1+(exp(-1*(testes_x1))))).astype(float)).tolist()
# df[0].apply(lambda x: float(x))
sigmoide_rescale = (8/(sigmoid_range/2))*0.8
multiplied_list = [(1/(1+(exp(-1*(element*sigmoide_rescale))))) for element in sigmoid_dx_list]
# multiplied_list
df_sigmoid['sigmoid']= multiplied_list
df_sigmoid.head()
# df_sigmoid

In [ ]:
unsigned_offset

## Sigmoid expansion (change 'x' rescale)

In [ ]:
# n_re_scale = (df_sigmoid['n'])*mult_factor
n_re_scale = (df_sigmoid['n'])*mult_factor
n_re_scale

In [ ]:
# df_sigmoid['n'] = (df_sigmoid['n'] + (unsigned_offset/2))*mult_factor
df_sigmoid['n'] = n_re_scale

df_sigmoid['n_zeros'] = (df_sigmoid['n'] + dx_sigmoid) #zeros para serem salvos na memória

In [ ]:
df_sigmoid

In [ ]:
# (pd.Series(sigmoid_dx)).plot()
(pd.Series(multiplied_list)).plot()

In [ ]:
df_sigmoid['sig_norm']= df_sigmoid['sigmoid']*norm_y
df_sigmoid

## df_sigmoid_bin

In [ ]:
#SIGMOID MEMORY
df_sigmoid_bin = pd.DataFrame()

#verification
df_sigmoid_bin['address'] = (df_sigmoid['n'])
df_sigmoid_bin['sigmoid'] = (df_sigmoid['sig_norm'])

#AQUI QUE O 'PONTO FIXO' DEVE COMEÇAR
# https://www.google.com/search?q=python+real+number+to+binary&oq=python+real+number+to+bin&aqs=chrome.1.69i57j0i22i30.8856j1j1&sourceid=chrome&ie=UTF-8
# https://stackoverflow.com/questions/51843297/convert-real-numbers-to-binary-and-vice-versa-in-python
# https://pythonhosted.org/bitstring/
df_sigmoid_bin['address_bin'] = (df_sigmoid['n'].apply(lambda x: float_to_binary(x ,m_bin,n_bin_sigmoid,is_signed)))
df_sigmoid_bin['address_zeros'] = (df_sigmoid['n_zeros'].apply(lambda x: float_to_binary(x ,m_bin,n_bin_sigmoid,is_signed)))

# df_sigmoid_bin['n_bin'] = (df_sigmoid['n'].apply(lambda x: float_to_binary(x ,m_bin,n_bin_sigmoid)))

df_sigmoid_bin['sigmoid_bin'] = (df_sigmoid['sig_norm'].apply(lambda x: float_to_binary(x ,m_bin,n_bin_sigmoid,is_sigmoid_signed)))
# df_sigmoid_bin.head()
df_sigmoid_bin

In [ ]:
# address_bin_converted = binary_to_float(df_sigmoid_bin['address_bin'], str(m_bin), str(n_bin_sigmoid))
address_bin_converted = df_sigmoid_bin['address_bin'].apply(lambda x: binary_to_float(x ,m_bin,n_bin_sigmoid,is_signed))
# address_bin_converted

In [ ]:
# (pd.Series(sigmoid_dx)).plot()
(pd.Series(address_bin_converted)).plot()

In [ ]:
#conferir quantas linhas de endereço duplicadas tem
len(df_sigmoid_bin['address_bin'])-len(df_sigmoid_bin['address_bin'].drop_duplicates())

In [ ]:
len(df_sigmoid_bin['address_bin'])

In [ ]:
duplicated_addr_bin = pd.DataFrame(df_sigmoid_bin['address_bin'].duplicated())

In [ ]:
duplicated_addr_bin.head()

In [ ]:
duplicated_addr_bin = duplicated_addr_bin[duplicated_addr_bin['address_bin'] == True]  # df will have only rows with True in c3

## Substitui máximos df_Sigmoid

In [ ]:
zeros_aux = str(np.zeros(m_bin+n_bin_sigmoid-1))
zeros_aux= zeros_aux.replace(".", "");
zeros_aux= zeros_aux.replace("[", "");
zeros_aux= zeros_aux.replace("]", "");
zeros_aux= zeros_aux.replace(" ", "");

ones_aux = str(np.ones(m_bin+n_bin_sigmoid-1))
ones_aux= ones_aux.replace(".", "");
ones_aux= ones_aux.replace("[", "");
ones_aux= ones_aux.replace("]", "");
ones_aux= ones_aux.replace(" ", "");

In [ ]:
##------------CORREÇÃO DE MÁXIMOS e MÍNIMOS BINÁRIOS------------##

#Substituindo erros de conversão float -> bin_2complement
#valores acima de valor x, estavam sendo convertidos com por ex, 100 sendo que
#o máximo seria 011 (apenas exemplo)

#--------------Address--------------# (signed)
df_sigmoid_bin['address_bin'].iloc[int(len(df_sigmoid_bin)/2):].replace(
    to_replace=['1' + zeros_aux],
    value='0' + ones_aux,
inplace=True)

if (is_sigmoid_signed == 0):

  #--------------Sigmoid--------------# (unsigned)
  df_sigmoid_bin['sigmoid_bin'].replace(to_replace=['10' + zeros_aux],
      value= '1' + ones_aux,
  inplace=True)

#============================================================================#
else:

    #--------------Sigmoid--------------# (signed)
  df_sigmoid_bin['sigmoid_bin'].replace(to_replace=['1' + zeros_aux],
      value= '0' + ones_aux,
  inplace=True)

df_sigmoid_bin

In [ ]:
#tirar linhas que contém valores binários com mais bits que o permitido
df_sigmoid_bin.drop(df_sigmoid_bin.tail(1).index,inplace=True) # drop last n rows
# df_sigmoid_bin.drop(df_sigmoid_bin.head(1).index,inplace=True) # drop first n rows    

# Graphs

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.offline as py

In [ ]:
pacitydef = 0.8
# teste = go.Scatter(
#     # x=dftf.xs('Date', level='Stock info', axis=1)[tickers[k]],
#     x= dftf.index,
#     y=y_test,
#     name="teste",
#     line= dict(color='#0080FF'),
#     opacity = opacitydef,
#     line_shape='spline'
#     )

sigmoid = go.Scatter(
    # x=dftf.xs('Date', level='Stock info', axis=1)[tickers[k]],
    x= df_sigmoid_bin['address'],
    y=df_sigmoid_bin['sigmoid'],
    name="sigmoid",
    line= dict(color='#FF8000'),
    opacity = 0.8,
    line_shape='spline')

dadospred = ([sigmoid])#, pred])
figpred = go.Figure(data=dadospred)#, layout=layout)
py.iplot(figpred)

In [ ]:
#https://stackoverflow.com/questions/35067304/pandas-dataframe-step-plot-where-post
# df_sigmoid_bin.plot(drawstyle="steps", linewidth=2, data='sigmoid')
plt.plot('address','sigmoid', drawstyle="steps", linewidth=2, data=df_sigmoid_bin, color ='orange')
# df_sigmoid_bin.plot(drawstyle="steps-post", linewidth=2)

In [ ]:
plt.plot( 'address', 'sigmoid', data=df_sigmoid_bin, linestyle='-', marker='o')